In [1]:
#import processing
import ds
#import ml
import model_saving
import TS
import TA
# Load the necessary packages and modules
#from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import numpy as np
import math as m
import talib
import pickle
from sklearn import svm

from hmmlearn.hmm import GaussianHMM
from sklearn.externals import joblib

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.


In [2]:
filename='/home/octo/Dropbox'+ '/SPY10.csv'

In [3]:
# Import a Kalman filter and other useful libraries
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import poly1d

def kalman_ma(data):
    x=data.Close
    # Construct a Kalman filter
    kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = x[1],
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.01)

    # Use the observed values of the price to get a rolling mean
    state_means, _ = kf.filter(x.values)
    state_means = pd.Series(state_means.flatten(), index=x.index)
    #df_ml['km']=state_means
    return state_means

In [48]:
# loading csv file
def get_csv_pd(path):
    spy_pd=pd.read_csv(path,sep=',',dtype={'askPrice':np.float32,'askSize':np.float32,
                                           'bidPrice':np.float32,'bidSize':np.float32},index_col=0,parse_dates=True)
    return spy_pd
def processing(df):
    df=df.drop(['Stock'], axis=1)
    df['Open']=((df.askPrice+df.bidPrice)/2)
    df['ABSR']=df.askSize/df.bidSize
    df['ABST']=(df.askSize+df.bidSize)
    #df['ABP']=df.Open-df.Open.rolling(6).mean()
    df.Volume=df.Volume.diff(6) 
    df['vwap']=((df.loc[:,'bidPrice']*df.loc[:,'bidSize'])+(df.loc[:,'askPrice']*df.loc[:,'askSize']))/(df.loc[:,'bidSize']+df.loc[:,'askSize'])
    df['OC']=df.Open-df.Close
    df['HL']=df.High-df.Low #if positive then market positive rather market negative
    df.Close=df.lastprice
    df['km']=df.Open-kalman_ma(df)
    df['UD']=np.where(np.logical_and(df.Open.diff(6)>0.005,df.km>0),1,np.where(np.logical_and(df.Open.diff(6)<-0.005,df.km<0),-1,0))
    df['U']=np.where(np.logical_and(df.Open.diff(6)>0.005,df.km>0),1,0)
    df['D']=np.where(np.logical_and(df.Open.diff(6)<-0.005,df.km<0),-1,0)
    df=df.drop(['askSize','bidSize','lastprice','askPrice','bidPrice','km'], axis=1)
    #df.ABD=df.ABD.diff(1)
    #df.OC=df.OC.diff(1)
    #df.HL=df.HL.diff(1)
    df.ABST=df.ABST/df.ABST.rolling(6).mean()
    df.vwap=df.vwap-df.Open.rolling(6).mean() 
    return df

def tech_ind(data):
    data['rsi']=talib.RSI(np.array(data.Close.astype('float64')),timeperiod=6)
    data.rsi=data.rsi/100
    data['atr']=talib.ATR(np.array(data.High.astype('float64')),np.array(data.Low.astype('float64')),np.array(data.Close.astype  ('float64')),timeperiod=6)
    data['mom']=talib.MOM(np.array(data.Close.astype('float64')), timeperiod=6)# mom consists 0
    #data.atr=data.atr-data.atr.rolling(6).mean()
    #data.mom=data.mom-data.mom.rolling(6).mean()
    data=data.drop(['Open','High','Close','Low'], axis=1)
    return data

In [49]:
data=get_csv_pd(filename)
data=processing(data)
data=tech_ind(data)
data=data[200:3500]

In [50]:
data=data.astype('float64')
data=data.dropna()

## State Analysis

In [52]:
hmm_loaded = joblib.load("markov_hmm.pkl")  
predictions_hmm = hmm_loaded.predict(data[['Volume','ABSR','ABST','vwap','OC','HL']])
data['h_state']=predictions_hmm

In [53]:
np.argmin(np.diag(hmm_loaded.transmat_))

0

In [54]:
data.groupby('h_state').size()

h_state
0    2864
1     436
dtype: int64

## SMOTE

## Bayes

#### Up

In [55]:
filename_nb = 'nbU.sav'
nb_loaded = pickle.load(open(filename_nb, 'rb'))
predictions_nb = nb_loaded.predict(data[['Volume','ABSR','ABST','vwap','OC','HL','rsi','atr','mom']])

In [56]:
data['prNBu']=predictions_nb

In [57]:
data.groupby('prNBu').size()

prNBu
0.00000    1664
1.00000    1636
dtype: int64

#### Dn

In [58]:
filename_nb = 'nbD.sav'
nb_loaded = pickle.load(open(filename_nb, 'rb'))
predictions_nb = nb_loaded.predict(data[['Volume','ABSR','ABST','vwap','OC','HL','rsi','atr','mom']])
data['prNBd']=predictions_nb

In [59]:
data.groupby('prNBd').size()

prNBd
-1.00000    1363
0.00000     1937
dtype: int64

In [60]:
data['prNB']=data.prNBu+data.prNBd

## outlier

In [61]:
filename_outlr = 'outlr.sav'
outlr_loaded = pickle.load(open(filename_outlr, 'rb'))
predictions_outlr = outlr_loaded.predict(data[['Volume','ABSR','ABST','vwap','OC','HL','rsi','atr','mom']])
data['outlr']=predictions_outlr

In [62]:
data.groupby('outlr').size()

outlr
-1     654
 1    2646
dtype: int64

## Predicting UD by RF

In [63]:
filename_rf = 'rf.sav'
rf_loaded = pickle.load(open(filename_rf, 'rb'))
predictions_rf = rf_loaded.predict(data[['Volume','ABSR','ABST','vwap','OC','HL','rsi','atr','mom']])

In [64]:
data['prUD']=predictions_rf

In [65]:
rf_loaded.score(data[['Volume','ABSR','ABST','vwap','OC','HL','rsi','atr','mom']],data.UD)

0.7121212121212122

In [66]:
data.groupby('prUD').size()

prUD
-1.00000     846
0.00000     1897
1.00000      557
dtype: int64

## Considering h_state and outlier data as  NAP. 

In [67]:
st=data.h_state==np.argmin(np.diag(hmm_loaded.transmat_))

In [88]:
data['UD']=data.prUD+data.prNB
data['UD']=np.where(data.UD==2,1,np.where(data.UD==-2,-1,0))
data['trade']=np.where(np.logical_or(np.logical_and(data.outlr==-1,data.UD==1),np.logical_and(st==True,data.UD==1)),1,np.where(np.logical_or(np.logical_and(data.outlr==-1,data.UD==-1),np.logical_and(st==True,data.UD==-1)),-1,0))
#data['trade']=np.where(np.logical_and(np.logical_and(data.outlr==-1,st==True),data.UD==1),1,np.where(np.logical_and(np.logical_and(data.outlr==-1,st==True),data.UD==-1),-1,0))

In [89]:
data.groupby('trade').size()

trade
-1     661
 0    2173
 1     466
dtype: int64

In [90]:
data.groupby('UD').size()

UD
-1     756
 0    2021
 1     523
dtype: int64

In [91]:
pd.crosstab(index=st,columns=data.UD)

UD,-1,0,1
h_state,,,
False,95,284,57
True,661,1737,466


In [84]:
pd.crosstab(index=data.outlr,columns=data.UD) 

UD,-1,0,1
outlr,,,
-1,341,283,30
1,415,1738,493


In [85]:
pd.crosstab(index=data.outlr,columns=st) 

h_state,False,True
outlr,,
-1,0,654
1,436,2210
